In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Pre and Post Session Surveys from AY18-19 (year1) and AY19-20 (year2)

year1_pre = pd.read_excel("Pre and post tests AY18–19/TaraProject_AY18–19Student Pre-Session Survey_ Academic Coaching  (Responses).xlsx")
year1_post = pd.read_excel("Pre and post tests AY18–19/TaraProject_AY18–19Student Post-Session Survey_ Academic Coaching  (Responses).xlsx")
year2_pre = pd.read_excel("Pre and post tests AY19–20/TaraProject_AY19–20Academic Coaching_ Pre-Session Student Survey  (Responses).xlsx")
year2_post = pd.read_excel("Pre and post tests AY19–20/TaraProject_AY19–20Academic Coaching_ Post-Session Student Survey  (Responses).xlsx")
pre_post = {"Pre1": year1_pre, "Post1": year1_post, "Pre2": year2_pre, "Post2": year2_post}

# Initial Consultations
initial_raw = pd.read_excel("TaraProject_AY19–20Initial Consultation (Intake) Form - AY 19-20 (Responses).xlsx")

# AC Roster Information
ac_roster = pd.read_excel("TaraProject_AY19–20AC Roster AY 19-20.xlsx", sheet_name = None)

In [ ]:
# Combine AC Rosters from both years

ac_roster1 = ac_roster['18-19 Photo Roster'].drop(columns=['Unnamed: 0', 'PHONE'])
ac_roster2 = ac_roster['19-20 roster'].drop(columns=['Unnamed: 14', 'AY 19-20 Wage', 'Phone Number',
                                                    'Expected Grad'])
ac_roster1 = ac_roster1.rename(columns = {'LAST NAME' : 'Last Name', 'PREFERRED NAME': 'Pref Name', 'First name': 
                                          'First Name','Class Level':'Class Rank'})

college = []
for i in range(len(ac_roster1['Major'])):
    L = ac_roster1['Major'][i].split('-')
    if(len(L) == 2):
        college += [L[0].strip().upper()]
        ac_roster1['Major'][i] = L[1].strip()
    else:
        college += ['NA']
ac_roster1['College'] = college
ac_roster1['Year'] = '18-19'
ac_roster2['Year'] = '19-20'
ac_roster_full = pd.merge(ac_roster1, ac_roster2, how='outer', on=['AndrewID', 'Last Name', 'First Name', 'Pref Name',
                                                 'Year', 'Major', 'College', 'Class Rank'], copy=False)
name = []
for i in range(len(ac_roster_full)):
    name += [ac_roster_full['First Name'][i] + ' ' + ac_roster_full['Last Name'][i]]
ac_roster_full['Full Name'] = name
ac_roster_full = ac_roster_full.drop(columns=['Last Name'])
ac_roster_full = ac_roster_full.append({'First Name': 'Michael', 'AndrewID':'mpoljak', 'Full Name': 'Michael Poljak',
                                       'Pref Name': 'Mike'},
                                      ignore_index=True)
ac_roster_full = ac_roster_full.append({'First Name': 'TC', 'AndrewID':'tce', 'Full Name': 'TC Eley'},
                                      ignore_index=True)

In [ ]:
# Change column names and drop columns that are irrelevant to data analysis

questions1, questions2 = [], []
for col in range(6,59):
    questions1 += [pre_post['Pre1'].columns[col]]
    pre_post['Pre1'] = pre_post['Pre1'].rename(columns={pre_post['Pre1'].columns[col] : col-5})
    pre_post['Post1'] = pre_post['Post1'].rename(columns={pre_post['Post1'].columns[col+1] : col-5})
for col in range(7,55):
    questions2 += [pre_post['Pre2'].columns[col]]
    pre_post['Pre2'] = pre_post['Pre2'].rename(columns={pre_post['Pre2'].columns[col] : col-6})
    pre_post['Post2'] = pre_post['Post2'].rename(columns={pre_post['Post2'].columns[col+1] : col-6})

# Pre 1:
pre_post['Pre1'] = pre_post['Pre1'].drop(columns = pre_post['Pre1'].columns[1:4])
pre_post['Pre1'] = pre_post['Pre1'].drop(columns = pre_post['Pre1'].columns[-5:])
pre_post['Pre1'] = pre_post['Pre1'].rename(columns={pre_post['Pre1'].columns[1] : 'AndrewID', 
                                              pre_post['Pre1'].columns[2] : 'Goals'})

# Pre 2:
pre_post['Pre2'] = pre_post['Pre2'].drop(columns = pre_post['Pre2'].columns[1:5])
pre_post['Pre2'] = pre_post['Pre2'].rename(columns={pre_post['Pre2'].columns[1] : 'AndrewID', 
                                              pre_post['Pre2'].columns[2] : 'Goals'})

# Post 1:
pre_post['Post1'] = pre_post['Post1'].drop(columns = pre_post['Post1'].columns[-12:])
pre_post['Post1'] = pre_post['Post1'].drop(columns = pre_post['Post1'].columns[1:5])
pre_post['Post1'] = pre_post['Post1'].rename(columns={pre_post['Post1'].columns[1] : 'AndrewID', 
                                              pre_post['Pre2'].columns[2] : 'Goals'})

# Post 2:
pre_post['Post2'] = pre_post['Post2'].drop(columns = pre_post['Post2'].columns[1:6])
pre_post['Post2'] = pre_post['Post2'].rename(columns={pre_post['Post2'].columns[1] : 'AndrewID'})


In [ ]:
pre_post['Post1'].iloc[43,1] = 'sallya'
pre_post['Post1'] = pre_post['Post1'].drop(index=32)

In [ ]:
# Identify students who have both pre-test and post-test data
students1 = pd.merge(pre_post['Pre1'], pre_post['Post1'], how='inner', on='AndrewID')['AndrewID'].drop_duplicates()
students2 = pd.merge(pre_post['Pre2'], pre_post['Post2'], how='inner', on='AndrewID')['AndrewID'].drop_duplicates()
students_complete = pd.merge(students1, students2, how='outer')

In [ ]:
print(len(pre_post['Pre1']), len(pre_post['Post1']), len(pre_post['Pre2']), len(pre_post['Post2']))
print(len(students1), len(students2), len(students_complete))

In [ ]:
# Drop students who do not have both pre-test and post-test data available
pre_post['Pre1'] = pd.merge(pre_post['Pre1'], students_complete, on='AndrewID')
pre_post['Post1'] = pd.merge(pre_post['Post1'], students_complete, on='AndrewID')
pre_post['Pre2'] = pd.merge(pre_post['Pre2'], students_complete, on='AndrewID')
pre_post['Post2'] = pd.merge(pre_post['Post2'], students_complete, on='AndrewID')
print(len(pre_post['Pre1']), len(pre_post['Post1']), len(pre_post['Pre2']), len(pre_post['Post2']))

In [ ]:
# Split dataset into subjective measures of student, subjective measures of AC/feedback, goals, and demographics

demographics = pd.concat([pre_post['Pre1'].iloc[:,[0,1,56,57,58,59,60]],
                pre_post['Pre2'].iloc[:,[0,1,53,54,55,56,57,58,59]]])
goals_pre = pd.concat([pre_post['Pre1'].iloc[:,0:3], pre_post['Pre2'].iloc[:,0:3]])
goals_post = pd.concat([pre_post['Post1'].iloc[:,0:3], pre_post['Post2'].iloc[:,0:3]])

subjective_measures_of_self = {'Pre': pd.concat([pd.concat([pre_post['Pre1'].iloc[:,[0,1]], pre_post['Pre1'].iloc[:,3:56]], axis=1),
                                               pd.concat([pre_post['Pre2'].iloc[:,[0,1]], pre_post['Pre2'].iloc[:,3:51]], axis=1)]),
                               'Post': pd.concat([pd.concat([pre_post['Post1'].iloc[:,[0,1]], pre_post['Post1'].iloc[:,3:56]], axis=1),
                                                pd.concat([pre_post['Post2'].iloc[:,[0,1]], pre_post['Post2'].iloc[:,4:51]], axis=1)])}

subjective_measures_of_AC = pd.concat([pd.concat([pre_post['Post1'].iloc[:,[0,1]], pre_post['Post1'].iloc[:,56:82]], axis=1),
                             pd.concat([pre_post['Post2'].iloc[:,[0,1]], pre_post['Post2'].iloc[:,53:]], axis=1)])

In [ ]:
initial = initial_raw.drop(columns=['Upload images of the session, if you have any', 'Other AD Services',
                               'External Resources', 'Unnamed: 18', 'Referrals [Academics]',
                               'STAFF Mentor', 'FACULTY Mentor','Referrals [Wellness]','Referrals [Health]', 
                                'Referrals'])
initial = initial.rename(columns={'Year in School:' : 'Year', 'Student Andrew ID' : 'AndrewID'})
initial = pd.merge(initial, students_complete, on='AndrewID')
len(initial)

In [ ]:
L1 = list(subjective_measures_of_AC.iloc[:,2])
for i in range(len(L1)):
    name = L1[i]
    if not (name.strip() in list(ac_roster_full['Full Name']) or name.strip() in list(ac_roster_full['First Name'])
           or name.strip() in list(ac_roster_full['Pref Name'])):
        print(i,name)

for i in range(len(L1)):
    name = L1[i].strip()
    for index,row in ac_roster_full.iterrows():
        if(name == row['Full Name'] or name == row['First Name'] or name == row['Pref Name']):
            L1[i] = row['AndrewID']
            
subjective_measures_of_AC['AC'] = L1

Current Dataframes:
- ac_roster_full (all AC info)
- pre_post dictionary for pre and post tests in 2018 and 2019 (4 dataframes) - raw
- students_complete (all students that have both pre and post test info)
- subjective_measures_of_self (subjective measures of student)
- subjective_measures_of_AC (subjective measures of AC/feedback)
- goals
- demographics
- initial (initial consultation info)

In [ ]:
ac_roster_full.to_csv("Dataframes/ac_roster.csv")

pre_post['Pre1'].to_csv("Dataframes/RAW/pre18.csv")
pre_post['Post1'].to_csv("Dataframes/RAW/post18.csv")
pre_post['Pre2'].to_csv("Dataframes/RAW/pre19.csv")
pre_post['Post2'].to_csv("Dataframes/RAW/post19.csv")

subjective_measures_of_self['Pre'].to_csv("Dataframes/subjective_measures_of_self_pre.csv")
subjective_measures_of_self['Post'].to_csv("Dataframes/subjective_measures_of_self_post.csv")

subjective_measures_of_AC.to_csv("Dataframes/subjective_measures_of_AC.csv")
goals_pre.to_csv("Dataframes/goals_pre.csv")
goals_post.to_csv("Dataframes/goals_post.csv")
demographics.to_csv("Dataframes/demographics.csv")

initial.to_csv("Dataframes/initial.csv")